In [ ]:
#@title Скачивание репозитория
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture
import os 

with capture.capture_output() as cap:
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
    print("[0;33mПодключаю...")
    drive.mount('/content/drive')
    mainpth="MyDrive"
clear_output()

if not os.path.exist("/content/drive/'+mainpth+'/SM"):
    %cd /content/drive/$mainpth/
    !git clone https://github.com/NeuroDonu/SM.git
else:
    %cd /content/drive/$mainpth/SM/
    !git pull
with capture.capture_output() as cap: 
    %cd /content/drive/$mainpth/SM
    !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
    !mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
    !wget https://developer.download.nvidia.com/compute/cuda/12.2.2/local_installers/cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
    !dpkg -i cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
    !cp /var/cuda-repo-ubuntu2204-12-2-local/cuda-*-keyring.gpg /usr/share/keyrings/
    !apt-get update
    !apt-get -y install cuda
    !rm -r /content/drive/MyDrive/SM/cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
clear_output()

with capture.capture_output() as cap:
    !apt-get install g++ gdb make ninja-build rsync zip
clear_output()

with capture.capture_output() as cap:
    %pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu
    %pip install -r requirements.txt
    %pip install -r requirements_cpu.txt
    %pip install opencv-python gradio moviepy
    %pip install pyngrok
clear_output()

with capture.capture_output() as cap:
    %cd /content/drive/$mainpth/SM/assets/pretrained_models
    !wget https://huggingface.co/NeuroDonu/refacer/blob/main/inswapper_128.onnx
    !wget https://huggingface.co/NeuroDonu/faceparser/blob/main/GFPGANv1.4.pth
    !wget https://huggingface.co/NeuroDonu/faceparser/blob/main/79999_iter.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x8.pth
    !wget https://huggingface.co/bluefoxcreation/Codeformer-ONNX/resolve/main/codeformer.onnx
clear_output()

inf('\u2714 Готово','success', '300px')

In [ ]:
#@title Запуск
from IPython.utils import capture
from IPython.display import clear_output
import time
import sys
import fileinput
from pyngrok import ngrok, conf
import re
import ipywidgets as widgets

Use_Cloudflare_Tunnel = False #@param {type:"boolean"}
#@markdown - Можешь включить, хз есть ли смысл

Ngrok_token = "" #@param {type:"string"}

share=''
if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token) , bind_tls=True).public_url

  for line in fileinput.input('/usr/local/lib/python3.10/dist-packages/gradio/blocks.py', inplace=True):
    if line.strip().startswith('self.server_name ='):
        line = f'            self.server_name = "{srv[8:]}"'
    if line.strip().startswith('self.protocol = "https"'):
        line = '            self.protocol = "https"'
    if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
        line = ''
    if line.strip().startswith('else "http"'):
        line = ''
    sys.stdout.write(line)
else:
  share='--colab'
!python /content/drive/$mainpth/SM/app.py $share --out_dir /content/results/